In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from matplotlib.lines import Line2D
from pandas.tseries.offsets import MonthEnd
from sklearn.metrics import mean_absolute_percentage_error as mean_ape
from sklearn.metrics import root_mean_squared_error as root_mse

sns.set(style="white")

In [ ]:
model_data_no_journal = pd.read_parquet("data/model_data_no_journal.parquet")
model_data_no_journal = model_data_no_journal[model_data_no_journal['political_alignment'] != 'autre']
model_data_no_journal['next_pres_votes_share'] = model_data_no_journal['pres_dummy'] * model_data_no_journal['pres_votes_share']

model_data = pd.read_parquet("data/model_data.parquet")
model_data = model_data[model_data['political_alignment'] != 'autre']
model_data['next_pres_votes_share'] = model_data['pres_dummy'] * model_data['pres_votes_share']

In [ ]:
presi_dates = [
    "26/04/1981", "10/05/1981",
    "24/04/1988", "08/05/1988",
    "23/04/1995", "07/05/1995",
    "21/04/2002", "05/05/2002",
    "22/04/2007", "06/05/2007",
    "22/04/2012", "06/05/2012",
    "21/04/2017", "07/05/2017",
    "10/04/2022", "24/04/2022"]
presi_months = pd.to_datetime(presi_dates, dayfirst=True).to_period('M').drop_duplicates()

legi_dates = [
    "14/06/1981", "21/06/1981",
    "16/03/1986",
    "05/06/1988", "11/06/1988",
    "21/03/1993", "28/03/1993",
    "25/05/1997", "01/06/1997",
    "09/06/2002", "16/06/2002",
    "10/06/2007", "17/06/2007",
    "10/06/2012", "17/06/2012",
    "11/06/2017", "18/06/2017",
    "12/06/2022", "19/06/2022",
    "29/06/2024", "06/07/2024"]
legi_months = pd.to_datetime(legi_dates, dayfirst=True).to_period('M').drop_duplicates()

regio_dates = [
    "16/03/1986",
    "22/03/1992",
    "15/03/1998",
    "21/03/2004", "28/03/2004",
    "14/03/2010", "21/03/2010",
    "06/12/2015", "13/12/2015",
    "20/06/2021", "27/06/2021"]
regio_months = pd.to_datetime(regio_dates, dayfirst=True).to_period('M').drop_duplicates()

canto_dates = [
    "14/03/1982", "21/03/1982",
    "10/03/1985", "17/03/1985",
    "25/09/1988", "02/10/1988",
    "20/03/1992", "27/03/1992",
    "15/03/1998", "22/03/1998",
    "11/03/2001", "18/03/2001",
    "21/03/2004", "28/03/2004",
    "9/03/2008", "16/03/2008",
    "20/03/2011", "27/03/2011"]
canto_months = pd.to_datetime(canto_dates, dayfirst=True).to_period('M').drop_duplicates()

dept_dates = [
    "22/03/2015", "29/03/2015",
    "20/06/2021", "27/06/2021"]
dept_months = pd.to_datetime(dept_dates, dayfirst=True).to_period('M').drop_duplicates()

muni_dates = [
    "06/03/1983", "13/03/1983",
    "12/03/1989", "19/03/1989",
    "11/06/1995", "18/06/1995",
    "11/03/2001", "18/03/2001",
    "09/03/2008", "16/03/2008",
    "23/03/2014", "30/03/2014",
    "15/03/2020", "28/06/2020"]
muni_months = pd.to_datetime(muni_dates, dayfirst=True).to_period('M').drop_duplicates()

europ_dates = [
    "13/06/1999",
    "13/06/2004",
    "07/06/2009",
    "25/05/2014",
    "26/05/2019",
    "09/06/2024"]
europ_months = pd.to_datetime(europ_dates, dayfirst=True).to_period('M').drop_duplicates()

main_elec_dates = presi_dates + legi_dates + europ_dates
main_elec_months = pd.to_datetime(main_elec_dates, dayfirst=True).to_period('M').drop_duplicates().sort_values()

other_elec_dates = regio_dates + canto_dates + dept_dates + muni_dates
other_elec_months = pd.to_datetime(other_elec_dates, dayfirst=True).to_period('M').drop_duplicates().sort_values()

def add_shaded_periods(ax_list, periods, color, alpha):
    start_period = None
    for i, period in enumerate(periods):
        if start_period is None:
            start_period = period
        is_last = (i == len(periods) - 1)
        is_gap = (not is_last and periods[i + 1] != period + 1)
        if is_last or is_gap:
            end_period = period
            start = start_period.to_timestamp()
            end = (end_period + MonthEnd(1)).to_timestamp()
            for ax in ax_list:
                ax.axvspan(start, end, color=color, alpha=alpha)
            start_period = None

Ce notebook utilise une approche économétrique particulière, ne cherchant pas à modéliser le plus précisément possible les comportements réels, mais étudiant leur écart à des comportements théoriques. Elle est intéressant ici car les activités journalistiques sont fortement normées : il s'agit donc de modéliser les comportements prescrits par ces normes afin de vérifier si elles sont respectées, et sinon, de quelle manière les journalistes y dérogent.

Considérons la répartition des citations entre les nuances politiques. Elle obéit théoriquement à une norme de représentativité, selon laquelle la couverture médiatique d'une nuance politique doit refléter son poids électoral et institutionnel. Soit $i$ une nuance politique : désignons par $Y_{i}$ la proportion des citations qu'elle reçoit, mesurons son poids électoral par son score aux dernières présidentielles (noté $P_{i}$) et son poids institutionnel par la proportion des sièges qu'elle détient à l'Assemblée Nationale (notée $AN_{i}$). La norme de représentativité prend alors la forme suivante :

$$Y_{i} = \alpha + \beta P_{i} + \gamma AN_{i}$$

Supposons que chaque nuance reçoit une fraction incompressible des citations, c'est-à-dire qu'elle bénéficie d'une couverture médiatique minimale même lorsque son poids électoral et institutionnel est nul. Cette hypothèse est plausible dans la mesure où les nuances politiques rassemblent plusieurs partis, et où l'un au moins de ces partis est toujours suffisamment important pour recevoir l'attention des journalistes. Ainsi :

$$\alpha = 0.01$$

(Cette constante permet aussi d'éviter des valeurs aberrantes dans le calcul des résidus).

Admettons également que le poids électoral des nuances politiques détermine les deux-tiers de leur couverture médiate, et leurs poids institutionnel le tiers restant. Cela donne :
$$\beta = 0.655$$
$$\gamma = 0.325$$

*NB 1 : comme les comportements théoriques priment dans l'analyse, c'est par rapport à eux que sont exprimés les résidus relatifs.*

*NB 2 : on parle de valeurs prescrites plutôt que prédites.*

In [ ]:
outcome = "quotes_share"

In [ ]:
y = model_data_no_journal[outcome]
y_norm = 0.01 + (0.655 * model_data_no_journal["na_share"]) + (0.325 * model_data_no_journal["pres_votes_share"])
mape = mean_ape(y, y_norm)
rmspe = root_mse(y, y_norm) / y_norm.mean()
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

Si cette norme était un modèle, ses performances prédictives ne seraient pas déraisonnables : la proportion des citations effectivement reçues par les nuances politiques s'écarte en moyenne de 70 % des valeurs prescrites. Si une nuance devrait recevoir 10 % des citations, elle en reçoit de 3 à 17 % en moyenne, ce qui correspond à des fluctuations assez modérées.

In [ ]:
X = model_data_no_journal[["na_share", "pres_votes_share"]]
X = sm.add_constant(X)
y = model_data_no_journal[outcome]
model = sm.OLS(y, X).fit(cov_type='HC3')

y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y_pred.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

En estimant directement ce modèle, les coefficients obtenus sont très proches ($\alpha = 0,015$ au lieu de $0,01$, $\beta = 0,73$ au lieu de $0,655$ et $\gamma = 0,29$ au lieu de $0,325$) et les performances prédictives également (avec une erreur quadratique moyenne 11 % plus faible mais une erreur moyenne 11,5 % plus élevée, ce qui signifie que le modèle estimé est plus efficace que le modèle *a priori* pour les valeurs extrêmes mais moins efficace pour les valeurs courantes). Cela montre que notre spécification est recevable, car elle correspond globalement aux tendances apparaissant dans les données.

In [ ]:
cutoff = pd.Period('2012-06', freq='M')
    
X = model_data_no_journal[model_data_no_journal["month"] < cutoff][["na_share", "pres_votes_share"]]
X = sm.add_constant(X)
y = model_data_no_journal[model_data_no_journal["month"] < cutoff][outcome]
model = sm.OLS(y, X).fit(cov_type='HC3')

y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y_pred.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

Ce résultat tient si on restreint l'estimation à la période antérieure à juin 2012, dans l'hypothèse où la norme de représentativité aurait prévalu nettement jusque là, mais se serait érodée ensuite. Les écarts avec le modèle *a priori* s'accentuent encore un peu, concernant aussi bien les coefficients que les performances prédictives, mais tous demeurent proches.

# 1. Ecarts à la norme de représentativité selon les nuances politiques

In [ ]:
model_data_no_journal['y_norm'] = 0.01 + (0.655 * model_data_no_journal["na_share"]) + (0.325 * model_data_no_journal["pres_votes_share"])
model_data_no_journal['rel_residuals'] = - 100 * (model_data_no_journal['y_norm'] - model_data_no_journal[outcome]) / model_data_no_journal['y_norm']

In [ ]:
summary = []

for alignment in model_data_no_journal['political_alignment'].unique():
    subset_data = model_data_no_journal[model_data_no_journal['political_alignment'] == alignment]
    y = subset_data[outcome]
    y_norm = subset_data['y_norm']
    mape = mean_ape(y, y_norm)
    rmspe = root_mse(y, y_norm) / y.mean()
    summary.append({
        'Political alignment': alignment,
        'MAPE': mape,
        'RMSPE': rmspe
    })

pd.DataFrame(summary).style.hide(axis=0)

*Commentaire...*

In [ ]:
cutoff1 = pd.Period('2002-06', freq='M')
cutoff2 = pd.Period('2017-06', freq='M')

models = {}

for period_label, period_filter in {
    '1981-2002': model_data_no_journal["month"] < cutoff1,
    '2002-2017': (model_data_no_journal["month"] >= cutoff1) & (model_data_no_journal["month"] < cutoff2),
    '2017-2024': model_data_no_journal["month"] >= cutoff2
}.items():
    
    models[period_label] = {}
    period_data = model_data_no_journal[period_filter]
    
    for alignment in period_data["political_alignment"].unique():
        models[period_label][alignment] = {}
        subset = period_data[period_data["political_alignment"] == alignment]

        if len(subset) >= 3:
            y = subset[outcome]
            y_norm = subset['y_norm']
            mape = mean_ape(y, y_norm)
            rmspe = root_mse(y, y_norm) / y.mean()

            models[period_label][alignment] = {
                "mape": mape,
                "rmspe": rmspe}

        else:
            models[period_label][alignment] = {
                "mape": None,
                "rmspe": None}

In [ ]:
nuances_order = ["Far right", "Right", "Center", "Left", "Far left"]

summary_list = []

for period, alignment_dict in models.items():
    for alignment, model_info in alignment_dict.items():
        summary_list.append({
            "period": period,
            "alignment": alignment,
            "mape": model_info["mape"],
            "rmspe": model_info["rmspe"]
        })

summary = pd.DataFrame(summary_list)
summary["alignment"] = pd.Categorical(summary["alignment"], categories=nuances_order, ordered=True)
summary = summary.pivot_table(index="alignment",
                                      columns="period",
                                      values=["mape", "rmspe"],
                                      observed=False)
summary = summary.reset_index()
summary.columns.names = [None, None]

format_cols = summary.columns[1:]
summary.style.hide(axis="index").format({col: "{:.3f}" for col in format_cols})

*Commentaire...*

In [ ]:
plot_data = model_data_no_journal.copy()
plot_data['month'] = plot_data['month'].dt.to_timestamp()

alignment_groups = [
    (['Far left', 'Far right'],
     {'Far left': 'crimson',
      'Far right': 'royalblue'}),
    (['Right', 'Left', 'Center'],
     {'Right': 'cornflowerblue',
      'Left': 'orchid',
      'Center': 'goldenrod'})]

In [ ]:
alignment = 'Far right'
subset_data = plot_data[plot_data['political_alignment'] == alignment].copy()
subset_data['MA_observed'] = subset_data[outcome].rolling(window=12).mean()
subset_data['MA_rel'] = subset_data['rel_residuals'].rolling(window=12).mean()

fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

sns.lineplot(data=subset_data, x='month', y=outcome, ax=axes[0], label='Observed values', alpha=0.15, color='crimson', linestyle='-')
sns.lineplot(data=subset_data, x='month', y='MA_observed', ax=axes[0], label='12 months moving average for OV', color='crimson', linestyle='dashdot')
sns.lineplot(data=subset_data, x='month', y='y_norm', ax=axes[0], label='Predicted values', color='teal', linestyle='dotted')
axes[0].set_title("Observed and Predicted Values")
axes[0].set_ylabel('')

sns.lineplot(data=subset_data, x='month', y='rel_residuals', ax=axes[1], label='Relative residuals', alpha=0.15, color='crimson', linestyle='-')
sns.lineplot(data=subset_data, x='month', y='MA_rel', ax=axes[1], label='12 months moving average for RR', color='crimson', linestyle='dashdot')
sns.lineplot(data=subset_data, x='month', y='y_norm', ax=axes[1], label='Predicted values', color='teal', linestyle='dotted')
axes[1].set_title("Relative Residuals (%)")
axes[1].set_xlabel('')
axes[1].set_ylabel('')

add_shaded_periods(axes, main_elec_months, color='black', alpha=0.1)

plt.suptitle("Proportion of Quotes Attributed to Far-Right Politicians")
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 12), sharex=True)

for ax, (political_alignments, colors) in zip(axes, alignment_groups):
    alignment_handles = []

    for alignment in political_alignments:
        subset_data = plot_data[plot_data['political_alignment'] == alignment].copy()
        subset_data['MA'] = subset_data[outcome].rolling(window=6).mean()
        
        ax.plot(subset_data['month'], subset_data[outcome], label=None,
                alpha=0.2, color=colors[alignment], linestyle='-')
        ax.plot(subset_data['month'], subset_data['MA'], label=None,
                alpha=0.65, color=colors[alignment], linestyle='dashdot')
        ax.plot(subset_data['month'], subset_data['y_norm'], label=None,
                alpha=1, color=colors[alignment], linestyle='dotted')
        
        alignment_handles.append(Line2D([0], [0], color=colors[alignment], lw=2, label=alignment))

    alignment_legend = ax.legend(handles=alignment_handles, title="Political alignment", loc="upper left")
    ax.add_artist(alignment_legend)

    line_type_handles = [
        Line2D([0], [0], color='black', lw=2, linestyle='-', label="Monthly average"),
        Line2D([0], [0], color='black', lw=2, linestyle='dashdot', label="6 months moving average"),
        Line2D([0], [0], color='black', lw=2, linestyle='dotted', label="Predictions")]
    ax.legend(handles=line_type_handles, title="Values", loc="upper right")

axes[-1].set_xlabel("")

add_shaded_periods(axes, main_elec_months, color='black', alpha=0.1)

plt.suptitle("Quote Distribution by Political Affiliation\nObserved vs. Predicted Values")
plt.tight_layout()
plt.savefig("output/nuances_val_graph.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 12), sharex=True)

for ax, (political_alignments, colors) in zip(axes, alignment_groups):
    alignment_handles = []

    for alignment in political_alignments:
        subset_data = plot_data[plot_data['political_alignment'] == alignment].copy()
        subset_data['MA'] = subset_data['rel_residuals'].rolling(window=6).mean()
        
        ax.plot(subset_data['month'], subset_data['rel_residuals'], label=None,
                alpha=0.2, color=colors[alignment], linestyle='-')
        ax.plot(subset_data['month'], subset_data['MA'], label=None,
                alpha=0.65, color=colors[alignment], linestyle='-.')
        ax.plot(subset_data['month'], [0] * len(subset_data), label=None,
                alpha=1, color='darkgray', linestyle=':')
        
        alignment_handles.append(Line2D([0], [0], color=colors[alignment], lw=2, label=alignment))

    alignment_legend = ax.legend(handles=alignment_handles, title="Political alignment", loc="upper left")
    ax.add_artist(alignment_legend)

    line_type_handles = [
        Line2D([0], [0], color='black', lw=2, linestyle='-', label="Monthly average"),
        Line2D([0], [0], color='black', lw=2, linestyle='-.', label="6 months moving average"),
        Line2D([0], [0], color='black', lw=2, linestyle=':', label="Predictions")
    ]
    ax.legend(handles=line_type_handles, title="Values", loc="upper right")

axes[-1].set_xlabel("")

add_shaded_periods(axes, main_elec_months, color='black', alpha=0.1)

plt.suptitle("Quote Distribution by Political Affiliation\nRelative Residuals (%)")
plt.tight_layout()
plt.savefig("output/nuances_res_graph.png", dpi=300, bbox_inches='tight')
plt.show()

On ne voit pas apparaître de tendance manifestement favorable à l'extrême droite.

# 2. Ecarts à la norme de représentativité selon les journaux

In [ ]:
model_data['y_norm'] = 0.01 + (0.655 * model_data["na_share"]) + (0.325 * model_data["pres_votes_share"])
model_data['rel_residuals'] = - 100 * (model_data['y_norm'] - model_data[outcome]) / model_data['y_norm']

In [ ]:
summary = []

for journal in model_data['journal'].unique():
    subset_data = model_data[model_data['journal'] == journal]
    y = subset_data[outcome]
    y_norm = subset_data['y_norm']
    mape = mean_ape(y, y_norm)
    rmspe = root_mse(y, y_norm) / y.mean()
    summary.append({
        'Journal': journal,
        'MAPE': mape,
        'RMSPE': rmspe})

pd.DataFrame(summary).style.hide(axis=0)

In [ ]:
plot_data = model_data.copy()
plot_data['month'] = plot_data['month'].dt.to_timestamp()

colors = {
    'Le Figaro': 'goldenrod',
    'Libération': 'limegreen',
    'Le Monde': 'orchid',
    'La Croix': 'skyblue',
    'Médiapart': 'crimson'}

alignments = [
    "Far right",
    "Right",
    "Center",
    "Left",
    "Far left"]

n_alignments = len(alignments)

In [ ]:
fig, axes = plt.subplots(n_alignments, 1, figsize=(16, 4 * n_alignments), sharex=True)

for i, alignment in enumerate(alignments):
    ax = axes[i]
    subset_data = plot_data[plot_data['political_alignment'] == alignment]

    for journal in subset_data['journal'].unique():
        if journal == 'Médiapart': continue
        sub_subset_data = subset_data[subset_data['journal'] == journal].copy()
        sub_subset_data['MA'] = sub_subset_data[outcome].rolling(window=12).mean()
        ax.plot(sub_subset_data['month'], sub_subset_data[outcome], label=journal,
                alpha=0.7, color=colors[journal], linestyle='-')

    ax.plot(subset_data['month'], subset_data['y_norm'], color='black', alpha=0.8, linestyle='dotted')
    ax.set_title(f"{alignment}")
    ax.legend()

add_shaded_periods(axes, main_elec_months, color='black', alpha=0.1)

plt.suptitle("""
Quote Distribution by Political Affiliation and Journal\n
Observed vs. Predicted Values
""")
plt.tight_layout()
plt.savefig("output/journals_val_graph.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(n_alignments, 1, figsize=(16, 4 * n_alignments), sharex=True)

for i, alignment in enumerate(alignments):
    ax = axes[i]
    subset_data = plot_data[plot_data['political_alignment'] == alignment]

    for journal in subset_data['journal'].unique():
        sub_subset_data = subset_data[subset_data['journal'] == journal].copy()
        sub_subset_data['MA'] = sub_subset_data['rel_residuals'].rolling(window=12).mean()
        ax.plot(sub_subset_data['month'], sub_subset_data['MA'], label=journal,
                alpha=0.7, color=colors[journal], linestyle='-')

    ax.plot(subset_data['month'], subset_data['y_norm'], color='black', alpha=0.8, linestyle='dotted')
    ax.set_title(f"{alignment}")
    ax.legend()

add_shaded_periods(axes, main_elec_months, color='black', alpha=0.1)

plt.suptitle("""
Quote Distribution by Political Affiliation and Journal\n
Relative Residuals (%) - 12 months moving averages
""")
plt.tight_layout()
plt.savefig("output/journals_res_graph.png", dpi=300, bbox_inches='tight')
plt.show()

Les comportements des différents journaux relativement à l'extrême droite s'avèrent très proches.